In [ ]:
import os, getpass

from langchain_core.callbacks import CallbackManagerForLLMRun, AsyncCallbackManagerForLLMRun
from langchain_core.outputs import ChatResult


def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

os.environ['OPENAI_API_KEY'] = os.environ.get('OPEN_AI_KEY')

In [ ]:
from langchain_openai import ChatOpenAI
gpt4o_chat = ChatOpenAI(model="gpt-4o", temperature=0)
gpt35_chat = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)

In [23]:
from langchain.chat_models.base import BaseChatModel
from langchain.schema import BaseMessage
from typing import List, Optional, Any, Dict
import requests

class ChatOllama(BaseChatModel):
    model_name: str = "deepseek-r1:14b"
    base_url: str = "http://localhost:11434"
    temperature: float = 0.0

    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        payload = {
            "model": self.model_name,
            "prompt": prompt,
            "max_tokens": 200,
            "temperature": self.temperature,
        }
        response = requests.post(f"{self.base_url}/api/generate", json=payload)
        response.raise_for_status()
        data = response.json()
        return data.get("response", "").strip()

    def _generate(
        self,
        messages: list[BaseMessage],
        stop: Optional[list[str]] = None,
        **kwargs: Any,
    ) -> Dict[str, Any]:
        def get_role(message: BaseMessage) -> str:
            if isinstance(message, HumanMessage):
                return "Human"
            elif isinstance(message, AIMessage):
                return "Assistant"
            else:
                return "Unknown"

        prompt = "\n".join([f"{get_role(message)}: {message.content}" for message in messages])
        output_text = self._call(prompt, stop=stop)
        return {"text": output_text}
    async def _agenerate(
        self,
        messages: list[BaseMessage],
        stop: Optional[list[str]] = None,
        **kwargs: Any,
    ) -> Dict[str, Any]:
        raise NotImplementedError("Async generation is not implemented for ChatOllama.")

    @property
    def _llm_type(self) -> str:
        return "ollama"

ollama_chat = ChatOllama(model_name="deepseek-r1:14b", temperature=0.0)

In [ ]:
from langchain_core.messages import HumanMessage

# Create a message
msg = HumanMessage(content="Say Hello", name="Adam")

# Create a message list
messages = [msg]

#Invoke a model with a list of messages
gpt4o_chat.invoke(messages)

In [24]:
from langchain.schema import HumanMessage, AIMessage

messages = [
    HumanMessage(content="Hello! Tell me a joke about the beach.", name="Adam"),
    AIMessage(content="Is this a response?", name="Ollama")
]
result = ollama_chat._generate(messages)
print("Ollama Chat Response:", result["text"])

JSONDecodeError: Extra data: line 2 column 1 (char 115)